In [1]:
import numpy as np  

class Chomsky:
    
    def __init__(self,file):
        
        def read_file(file):
            Lines =  open(file, 'r').readlines() 
            for line in Lines:
                self.transitions.append( {'s': line[0],  'd': line[1:].split()[1]  } ) 
                
        # Initializing the transitions list
        self.transitions = list() 
        read_file(file)
        
        
    def elminate_empty(self):
        
        def get_empty_elements():
            empty_arr = list()  
            k = 0 
            for i in range(len(self.transitions)):
                if self.transitions[i - k ].get('d') == '@':
                    empty_arr.append(self.transitions[i - k].get('s')) 
                    self.transitions.pop(i-k) 
                    k += 1    
            return empty_arr 
        
        
        # Initiliazing an uptaded list of replacing the empty transitions
        _updated = list()  
        
        # replacing non terminals that derive in empty 
        for empty_elem in get_empty_elements():
            for i in range(len(self.transitions)): 
                if empty_elem in self.transitions[i].get('d') : 

                    freq_empty =  self.transitions[i].get('d').count(empty_elem)  

                    if freq_empty > 1:

                        # getting the indexes array of where the letter is met  
                        index_arr = list()
                        for j in range(len(self.transitions[i].get('d'))):
                            if  empty_elem == self.transitions[i].get('d')[j]:
                                index_arr.append(j)  
            


                        def return_word(word,bin_arr):
                            k = 0
                            for j in range(len(bin_arr)):
                                if bin_arr[j] == 0: 
                                    word = word[:index_arr[j]- k ] + word[index_arr[j] - k +1:]  
                                    k+=1
                                    #print(word)
                            return word   


                        def binary_strings(n, arr, indx):
                            if indx == n:
                                #print(arr)
                                if arr != [1]*freq_empty: 
                                    a = {'s' :  self.transitions[i].get('s'), 'd':  return_word( self.transitions[i].get('d'),arr) }
                                    if a not in _updated:
                                        _updated.append(a)
                                return 

                            arr[indx] = 0
                            binary_strings(n, arr, indx + 1)
                            arr[indx] = 1 
                            binary_strings(n, arr, indx + 1)    


                        # 
                        bin_arr  = [None]*freq_empty  
                        binary_strings(freq_empty, bin_arr, 0)

                    else:
                        _updated.append({'s' : self.transitions[i].get('s'), 'd' : self.transitions[i].get('d').replace(empty_elem, '') })  
                        
        
     
        self.transitions.extend(_updated)
    
    
    def eliminate_renamings(self):
        
        def get_renamings_list():
            renaming_list = list() 
            k = 0  
            for i in range (len(self.transitions)):
                if self.transitions[i-k].get('s').isupper()  and self.transitions[i-k].get('d').isupper() and len( self.transitions[i-k].get('s')) == 1 and len( self.transitions[i-k].get('d')) == 1:
                    if [self.transitions[i-k].get('s'), self.transitions[i-k].get('d') ] not in renaming_list:
                        renaming_list.append( [ self.transitions[i-k].get('s'),  self.transitions[i-k].get('d') ] ) 
                        self.transitions.pop(i-k) 
                        k += 1 
                            
            return renaming_list   
        
        def update_renamings_list(): 
            # Calling the above function to get the non-term arr Example: S-> B 
            renaming_list = get_renamings_list()  

            # Checking for multiple derivations. Adding them to the list
            for i in range(len(renaming_list)):
                for ren in renaming_list:
                    if renaming_list[i][0] == ren[1] and  ren[0] !=renaming_list[i][1] and [ ren[0],  renaming_list[i][1] ] not in renaming_list:
                        renaming_list.append([ ren[0],  renaming_list[i][1] ])
            return renaming_list 

        _updated = list() 
        for ren in update_renamings_list(): 
            for elem in self.transitions: 
                if elem.get('s') == ren[1]:
                    if dict({'s': ren[0], 'd': elem.get('d') }) not in self.transitions:
                        _updated.append( {'s': ren[0], 'd' : elem.get('d')} ) 
        

        self.transitions.extend(_updated) 



    def  eliminate_non_productive(self):

         
        # Initializing the dict with nonterminals and bool value 
        prod_dict = dict()
        for nont in   list( np.unique( [ x.get('s') for x in self.transitions] )) :
            prod_dict.update({nont : False})  
        
        # First updating the bool dictionay with nonterminals that surely derive in a terminal Ex: A-> b
        for elem in self.transitions:
            if elem.get('d').islower() and  prod_dict[elem.get('s')] == False  :
                prod_dict[elem.get('s')] = True 
        
        
        # Updating the dict with parsing the strings of destionation of transitions 
        def recursive_prod():
            for elem in self.transitions:
                len_dest = len(elem.get('d')) 
                for l in elem.get('d'): 
                    if l.islower() or (l.isupper() and prod_dict[l] == True ) :
                        len_dest -= 1 
                        continue 
                if len_dest == 0 and prod_dict[elem.get('s')] == False:
                    prod_dict[elem.get('s')] = True
                    print('AUF') 
                    return True
            
            return False
                      
        index_prod = True            
        while index_prod:
            index_prod = recursive_prod()
            
        
        # Eliminating the non productive 
        k = 0
        for i  in range(len(self.transitions)):
            if prod_dict[self.transitions[i-k].get('s')] == False:
                self.transitions.pop(i-k)
                k += 1 
                continue
            for l in self.transitions[i-k].get('d'):
                if l.isupper()  and prod_dict[l] == False:
                    self.transitions.pop(i-k)
                    k += 1
                    continue  


    def eliminate_non_access(self): 

        # Initializing the dict with nonterminals and bool value 
        innac_dict = dict()
        for nont in   list( np.unique( [ x.get('s') for x in self.transitions] )) :
            innac_dict.update({nont : True}) 


        # Updating the innacassible dict 
        for elem in self.transitions:
            for l in elem.get('d'):
                if l.islower():
                    continue 
                if l.isupper() and innac_dict[l] == True:
                    innac_dict[l] = False 
        
        # Removing the innaccessible items.
        k = 0
        for i in range(len(self.transitions)):
            if  innac_dict [ self.transitions[i-k].get('s') ] == True:
                self.transitions.pop(i-k)
                k += 1


    
    def to_chomsky(self): 
        
        term_dict = dict() 
        it_x = 0  
        _chomsky = list()
        
        # Initializing and adding already Chomsky form symbols to the _chomsky array. 
        k = 0 
        for i in range(len(self.transitions)):
            if   len( self.transitions[i-k].get('d') ) == 1 and self.transitions[i-k].get('d').islower() : 
                _chomsky.append(self.transitions[i-k])   
                #print(self.transitions[i-k])  
                if self.transitions[i-k].get('d') not in term_dict.keys(): 
                    it_x += 1
                    term_dict[self.transitions[i-k].get('d')] = 'X' + str(it_x) 
                self.transitions.pop(i-k)  
                k +=1  
            if len( self.transitions[i-k].get('d') ) == 2  and self.transitions[i-k].get('d')[0].isupper() and self.transitions[i-k].get('d')[1].isupper():
                _chomsky.append(self.transitions[i-k])  
                self.transitions.pop(i-k)  
                k +=1    

        # Adding all nonterminals to the chomsky_dict in order to replace them later. 
        for i in range(len(self.transitions)):
            for l in self.transitions[i].get('d'):
                if l.islower() and l not in term_dict.keys(): 
                    it_x += 1
                    term_dict[l] = 'X' + str(it_x)  
        


        def check_chomsky_form(w): 
            counter = 0
            for l in w:
                if l.isalpha():
                    counter += 1
            if counter == 2:
                return True 
            return False 
    
        # Replacing terminals met with Xs. 
        k = 0 
        for i in range(len(self.transitions)):
            for l in self.transitions[i-k].get('d'):
                if l.islower(): 
                    self.transitions[i-k]['d'] = self.transitions[i-k].get('d').replace(l,term_dict[l]) 
                    if check_chomsky_form( self.transitions[i-k].get('d')):
                        _chomsky.append(self.transitions[i-k]) 
                        self.transitions.pop(i-k)
                        k +=1   
         
        # The logic 
        possible_combinations = dict()  
        it_Y = 0
        
        def check_digit_index(l):
            if l.isdigit():
                return 1
            else:
                return 0 


        def check_unique(w,c):
            for elem in possible_combinations.keys():
                for l in elem:
                    if l in w and c <= possible_combinations[elem]:
                        return False 
            return True 
        

        def add_freq(w): 
            counter = 0 
            for elem in self.transitions:
                if w in elem.get('d') and w != elem.get('d'):
                    counter += 1 
            #print(w)
            if w not in possible_combinations.keys() and counter > 1 and check_unique(w,counter): 
                possible_combinations[w] = counter  


        def get_possible_combinations(w,init):   
        
            if len(w) - check_digit_index(w[-1]) <= 2  :
                return     
            
            if init == False:
                add_freq(w)

            for i in range(2 +  check_digit_index(w[1]), len(w) - check_digit_index(w[-1])): 
                if w[i].isdigit():
                    continue 
                
                #print(w[:i]) 
                add_freq(w[:i]) 
                
            get_possible_combinations(w[ check_digit_index(w[1]) + 1:],False) 

        
        #MAIN       
        while True:
            if bool(self.transitions) == False:
                break 
                
            possible_combinations = dict() 

            for elem in self.transitions:
                #print(elem)
                get_possible_combinations(elem.get('d'), True) 
                
            for key in possible_combinations.keys():
                it_Y +=1
                term_dict[key] =  f'Y{it_Y}' 
                
            
            k = 0 
            for i in range(len(self.transitions)):
                for key in possible_combinations.keys():
                    if key in self.transitions[i-k].get('d'):
                        self.transitions[i-k]['d'] =  self.transitions[i-k].get('d').replace(key, term_dict[key])   

                if  check_chomsky_form(self.transitions[i-k].get('d')):
                    _chomsky.append(self.transitions[i-k])
                    self.transitions.pop(i-k)
                    k += 1 

        for i,j in  zip(term_dict.values(), term_dict.keys()): 
            _chomsky.append( {'s': i, 'd':j} ) 

        return _chomsky

        
        
    def run(self):
        print('Initial: \n')
        print(self.transitions) 
        print('\n Empty symbols elimination: \n')
        self.elminate_empty()     
        print(self.transitions)
        print('\n Renamings elimination: \n')
        self.eliminate_renamings() 
        print(self.transitions)    
        print('\n Elimination of non accessible items: \n')
        self.eliminate_non_access() 
        print(self.transitions)
        print('\n Elimanation of non productive \n')
        self.eliminate_non_productive()
        print(self.transitions)   
        print('\n Chomsky Final: \n')  
        print(self.to_chomsky())
        

In [2]:
c = Chomsky('test.txt')

In [3]:
c.run()

Initial: 

[{'s': 'S', 'd': 'AC'}, {'s': 'S', 'd': 'bA'}, {'s': 'S', 'd': 'B'}, {'s': 'S', 'd': 'aA'}, {'s': 'A', 'd': '@'}, {'s': 'A', 'd': 'aS'}, {'s': 'A', 'd': 'ABAb'}, {'s': 'B', 'd': 'a'}, {'s': 'B', 'd': 'AbSA'}, {'s': 'C', 'd': 'abC'}, {'s': 'D', 'd': 'AB'}]

 Empty symbols elimination: 

[{'s': 'S', 'd': 'AC'}, {'s': 'S', 'd': 'bA'}, {'s': 'S', 'd': 'B'}, {'s': 'S', 'd': 'aA'}, {'s': 'A', 'd': 'aS'}, {'s': 'A', 'd': 'ABAb'}, {'s': 'B', 'd': 'a'}, {'s': 'B', 'd': 'AbSA'}, {'s': 'C', 'd': 'abC'}, {'s': 'D', 'd': 'AB'}, {'s': 'S', 'd': 'C'}, {'s': 'S', 'd': 'b'}, {'s': 'S', 'd': 'a'}, {'s': 'A', 'd': 'Bb'}, {'s': 'A', 'd': 'BAb'}, {'s': 'A', 'd': 'ABb'}, {'s': 'B', 'd': 'bS'}, {'s': 'B', 'd': 'bSA'}, {'s': 'B', 'd': 'AbS'}, {'s': 'D', 'd': 'B'}]

 Renamings elimination: 

[{'s': 'S', 'd': 'AC'}, {'s': 'S', 'd': 'bA'}, {'s': 'S', 'd': 'aA'}, {'s': 'A', 'd': 'aS'}, {'s': 'A', 'd': 'ABAb'}, {'s': 'B', 'd': 'a'}, {'s': 'B', 'd': 'AbSA'}, {'s': 'C', 'd': 'abC'}, {'s': 'D', 'd': 'AB'},